# Original Objective function
$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} k_{ij} q_j \prod_{i' \in \{1, \ldots, m\}}^{\text{msg}} \left( k_{i'j} p_{i'} + 0^{k_{i'j}} \right)
$$

### Simplification assumption
We assume probobility of recieving a tag $q_{j}$ is equal to probobility of recieving message $p_{i}$  
$$q_{j} = p_{i} = p$$

### C1 : Using all tags
Each tag must verify at least one message
$$
\sum_{i=1}^{m} k_{ij} \geq 1 \quad \text{for each } j \in \{1, 2, \ldots, \text{|T|}\}
$$
### C2: Using all messages
Each message must be signed with at least one tag
$$
\sum_{j=1}^{n} k_{ij} \geq 1 \quad \text{for each } i \in \{1, 2, \ldots, \text{|M|}\}
$$

### C3: Define auxiliary varible for number of messages each tag signs
we define number of messages that each tag signs as an auxiliray variable to use later in transforming objective function to a linear model
$$
z_j = \sum_{i=1}^{n} k_{ij} \quad \text{for each } j \in \{1, 2, \ldots, \text{tags}\}
$$
$$
z_j \geq 1 \quad \text{for each } j \in \{1, 2, \ldots, \text{tags}\}
$$


### C4: Re-writing the product part in linear
After simplifying all probobilities to be equal the product part could be linearized using some auxiliary variable and constraints as follow:

$$
\prod_{i' \in \{1, \ldots, m\}}^{\text{msg}} \left( k_{i'j} p_{i'} + 0^{k_{i'j}} \right) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} \sum_{i' \in \{1, \ldots, m\}}^{\text{msg}} w_{ji'} p^j
$$

$$
y_{j} = \sum_{i' \in \{1, \ldots, m\}}^{\text{msg}} w_{ji'}   \quad for \quad j \in \{1,...,n\}
$$
$$w_{jy} \in \{0,1\}$$
Now the next constraint should be define to force the correcponding $w_{ji'} = 1$ which can be done using another auxiliary variable and two constraints as follow
$$z_{j} - i \leq M y_{j}$$
$$i - z_{j} \leq M (1-y_{j})$$
$$ i \in \quad \{0,...,m\}$$


## Simplified Objective function
With linearizing product part, the objective function could be rewritten as: 

$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} k_{ij} p \text{ } *\sum_{j \in \{1, \ldots, n\}}^{\text{tag}} y_{j} p^j \quad
$$
or even more simple as
$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} k_{ij}\text{ } y_{j}\text{ } p^{(j+1)}

$$

The final part is linearizing the product of the two binary variables $k_{ij}w_{j}$ which could be done this way:

$$x_{ij} \leq k_{ij} $$
$$x_{ij} \leq y_{j} $$
$$x_{ij} \geq k_{ij} + y_{j} -1 $$
$$x_{ij}\in \{0,1\}$$

# Linear Objective function
$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} x_{ij}\text{ } p^{(j+1)}$$

In [ ]:
import numpy as np
import pandas as pd
from pulp import *

In [ ]:

def max_expected_A(n_tags, m_msgs, p):
   
    # Create a PuLP problem with maximization
    prob = LpProblem("Maximize_Expected_A", LpMaximize)

    # Define the binary variables k indexed by (i, j)
    k = {(i, j): LpVariable(f'k_{i}_{j}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1)}

    # Define the binary variables w indexed by (j,i)
    w = {(j, i): LpVariable(f'w_{j}_{i}', cat='Binary') for j in range(1, n_tags+1) for i in range(1, m_msgs+1)}

    # Define the binary variables x indexed by (i, j)
    x = {(i, j): LpVariable(f'x_{i}_{j}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1)}

    # Define the binary variables w indexed by j
    y = {j: LpVariable(f'y_{j}', cat='Binary') for j in range(1, n_tags+1)}

    # Define the continuous variables z indexed by j
    z = {j: LpVariable(f'z_{j}', cat='Integer', lowBound=0) for j in range(1, n_tags+1)}

    # # Constraints
    # for j in range(1, n_tags+1):
    #     prob += lpSum(k[i, j] for i in range(1, m_msgs+1)) >= 1, f"Tag_{j}_constraint"

    # for i in range(1, m_msgs+1):
    #     prob += lpSum(k[i, j] for j in range(1, n_tags+1)) >= 1, f"Message_{i}_constraint"

    for j in range(1, n_tags+1):
        prob += z[j] == lpSum(k[i, j] for i in range(1, m_msgs+1)), f"Z_{j}_definition"
        prob += z[j] >= 1, f"Z_{j}_non_zero"
        
    #for j in range(1, n_tags+1):
    #    prob += lpSum(w[j, i] for i in range(1, m_msgs+1)) >= 1, f"Sum_w_{j}_equals_1"

    for j in range(1, n_tags+1):
        for i in range(1, m_msgs+1):
            prob += z[j] - i <= M * y[j], f"BigM_1_{j}_{i}"
            prob += i - z[j] <= M * (1 - y[j]), f"BigM_2_{j}_{i}"

    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            prob += x[i, j] <= k[i, j], f"x_{i}_{j}_leq_k_{i}_{j}"
            prob += x[i, j] <= y[j], f"x_{i}_{j}_leq_w_{j}"
            prob += x[i, j] >= k[i, j] + y[j] - 1, f"x_{i}_{j}_geq_k_{i}_{j}_plus_w_{j}_minus_1"


    # Objective Function
    prob += lpSum(x[i, j] * P[j+1] for i in range(1, m_msgs+1) for j in range(1, n_tags+1)), "Maximize_E_A"

    # Solve the problem
    prob.solve()

    # Extract the optimal values of k_{ij}
    k_values = np.zeros((m_msgs, n_tags), dtype=int)
    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            k_values[i-1, j-1] = value(k[i, j])

    k_df = pd.DataFrame(k_values, columns=[f'k_{j}' for j in range(1, n_tags+1)], index=[f'Msg_{i}' for i in range(1, m_msgs+1)])
    k_df.columns = [f't{i}' for i in range(1,n_tags+1)]
    k_df.index = [f'm{i}' for i in range(1,m_msgs+1)]
    print(k_df)

    # Print the variable values
    for v in prob.variables():
        print(f"{v.name} = {v.varValue}")

    # Print the objective value
    print(f"Objective value: {value(prob.objective)}")
    
    return k_df


In [20]:
n_tags = 6
n_msgs = 6
p = 0.001 
P = {j: p**j for j in range(1, n_tags+2)}  # p^j coefficients
M = 1000     # A sufficiently large number for the constraints

In [21]:
max_expected_A(n_tags, n_msgs, p)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/cb18d64b8acc47cfa278898605f6a143-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/cb18d64b8acc47cfa278898605f6a143-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 197 COLUMNS
At line 846 RHS
At line 1039 BOUNDS
At line 1124 ENDATA
Problem MODEL has 192 rows, 84 columns and 444 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6e-06 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 259 substitutions
Cgl0004I processed model has 48 rows, 42 columns (42 integer (42 of which binary)) and 126 elements
Cbc0038I Initial state - 0 integers unsatisfied

,t1,t2,t3,t4,t5,t6
m1,1,1,1,1,1,1
m2,1,0,0,0,0,0
m3,1,0,0,0,0,0
m4,1,0,0,0,0,0
m5,1,0,0,0,0,0
m6,1,0,0,0,0,0


In [19]:
print(6*P[6])
print(6*p)

5.648880896406
5.9399999999999995
